In [1]:
from flask import Flask, render_template, request
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
from sklearn.metrics import classification_report
import re
import string
from sklearn.model_selection import cross_val_score


In [2]:
app = Flask(__name__)

In [3]:
data_fake = pd.read_csv('Fake.csv')
data_true = pd.read_csv('True.csv')

In [4]:
data_fake.head()

,title,text,subject,date
0,Donald Trump Sends Out Embarrassing New Year’...,Donald Trump just couldn t wish all Americans ...,News,"December 31, 2017"
1,Drunk Bragging Trump Staffer Started Russian ...,House Intelligence Committee Chairman Devin Nu...,News,"December 31, 2017"
2,Sheriff David Clarke Becomes An Internet Joke...,"On Friday, it was revealed that former Milwauk...",News,"December 30, 2017"
3,Trump Is So Obsessed He Even Has Obama’s Name...,"On Christmas day, Donald Trump announced that ...",News,"December 29, 2017"
4,Pope Francis Just Called Out Donald Trump Dur...,Pope Francis used his annual Christmas Day mes...,News,"December 25, 2017"


In [5]:
data_fake["class"]= 0
data_true["class"]= 1


In [6]:
data_fake.shape, data_true.shape

((23481, 5), (21417, 5))

In [7]:
data_fake_manual_testing = data_fake.tail(10)
for i in range(23480,23470,-1):
     data_fake.drop([i] , axis = 0, inplace =True)

data_true_manual_testing = data_true.tail(10)
for i in range(21416,21406,-1):
     data_true.drop([i] , axis = 0, inplace =True)


In [8]:
data_fake.shape, data_true.shape

((23471, 5), (21407, 5))

In [9]:
data_fake_manual_testing['class'] = 0
data_true_manual_testing['class'] = 1

C:\Users\Asus\AppData\Local\Temp\ipykernel_20672\277247672.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data_fake_manual_testing['class'] = 0
C:\Users\Asus\AppData\Local\Temp\ipykernel_20672\277247672.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data_true_manual_testing['class'] = 1


In [10]:
data_fake_manual_testing.head(10)

,title,text,subject,date,class
23471,Seven Iranians freed in the prisoner swap have...,"21st Century Wire says This week, the historic...",Middle-east,"January 20, 2016",0
23472,#Hashtag Hell & The Fake Left,By Dady Chery and Gilbert MercierAll writers ...,Middle-east,"January 19, 2016",0
23473,Astroturfing: Journalist Reveals Brainwashing ...,Vic Bishop Waking TimesOur reality is carefull...,Middle-east,"January 19, 2016",0
23474,The New American Century: An Era of Fraud,Paul Craig RobertsIn the last years of the 20t...,Middle-east,"January 19, 2016",0
23475,Hillary Clinton: ‘Israel First’ (and no peace ...,Robert Fantina CounterpunchAlthough the United...,Middle-east,"January 18, 2016",0
23476,McPain: John McCain Furious That Iran Treated ...,21st Century Wire says As 21WIRE reported earl...,Middle-east,"January 16, 2016",0
23477,JUSTICE? Yahoo Settles E-mail Privacy Class-ac...,21st Century Wire says It s a familiar theme. ...,Middle-east,"January 16, 2016",0
23478,Sunnistan: US and Allied ‘Safe Zone’ Plan to T...,Patrick Henningsen 21st Century WireRemember ...,Middle-east,"January 15, 2016",0
23479,How to Blow $700 Million: Al Jazeera America F...,21st Century Wire says Al Jazeera America will...,Middle-east,"January 14, 2016",0
23480,10 U.S. Navy Sailors Held by Iranian Military ...,21st Century Wire says As 21WIRE predicted in ...,Middle-east,"January 12, 2016",0


In [11]:
data_true_manual_testing.head(10)

,title,text,subject,date,class
21407,"Mata Pires, owner of embattled Brazil builder ...","SAO PAULO (Reuters) - Cesar Mata Pires, the ow...",worldnews,"August 22, 2017",1
21408,"U.S., North Korea clash at U.N. forum over nuc...",GENEVA (Reuters) - North Korea and the United ...,worldnews,"August 22, 2017",1
21409,"U.S., North Korea clash at U.N. arms forum on ...",GENEVA (Reuters) - North Korea and the United ...,worldnews,"August 22, 2017",1
21410,Headless torso could belong to submarine journ...,COPENHAGEN (Reuters) - Danish police said on T...,worldnews,"August 22, 2017",1
21411,North Korea shipments to Syria chemical arms a...,UNITED NATIONS (Reuters) - Two North Korean sh...,worldnews,"August 21, 2017",1
21412,'Fully committed' NATO backs new U.S. approach...,BRUSSELS (Reuters) - NATO allies on Tuesday we...,worldnews,"August 22, 2017",1
21413,LexisNexis withdrew two products from Chinese ...,"LONDON (Reuters) - LexisNexis, a provider of l...",worldnews,"August 22, 2017",1
21414,Minsk cultural hub becomes haven from authorities,MINSK (Reuters) - In the shadow of disused Sov...,worldnews,"August 22, 2017",1
21415,Vatican upbeat on possibility of Pope Francis ...,MOSCOW (Reuters) - Vatican Secretary of State ...,worldnews,"August 22, 2017",1
21416,Indonesia to buy $1.14 billion worth of Russia...,JAKARTA (Reuters) - Indonesia will buy 11 Sukh...,worldnews,"August 22, 2017",1


In [44]:
data_merge = pd.concat([data_fake,data_true],axis = 0)
data_merge.sample(10)

,title,text,subject,date,class
17900,POWERFUL LETTER To Cowardly Steelers Coach Wil...,"Dear Mike Tomlin, James Harrison, Ben Roethlis...",left-news,"Sep 28, 2017",0
5144,New Interior chief vows review of Obama-era ru...,WASHINGTON (Reuters) - New Interior Secretary ...,politicsNews,"March 3, 2017",1
18077,Conservative plotters told to get behind UK's ...,LONDON (Reuters) - Lawmakers in British Prime ...,worldnews,"October 7, 2017",1
16465,BAM! RUDY GIULIANI Does A Number On Political ...,Rudy is the best! He stays on message and was ...,Government News,"Aug 10, 2016",0
15596,China slams Indian minister's visit to dispute...,BEIJING (Reuters) - China on Monday criticized...,worldnews,"November 6, 2017",1
22402,"It’s More Likely That a US Insider, Not Russia...",By Vin ArmaniHillary Clinton continues to bla...,US_News,"November 5, 2016",0
20087,"In a first, Myanmar's 'ethnic cleansing' unite...",YANGON (Reuters) - If there s one thing that u...,worldnews,"September 14, 2017",1
9683,Pot legalization supporters launch California ...,LOS ANGELES (Reuters) - Supporters of a drive ...,politicsNews,"May 5, 2016",1
8678,How Wounded Warrior Project Treated A Triple-...,This just goes to show that you should always ...,News,"January 18, 2016",0
21106,"Iran says jailed U.S. student, dual nationals ...",DUBAI (Reuters) - Iran confirmed on Sunday tha...,worldnews,"September 3, 2017",1


In [13]:
data_merge.columns

Index(['title', 'text', 'subject', 'date', 'class'], dtype='object')

In [14]:
data = data_merge.drop(['title','subject','date'],axis = 1)  

In [15]:
data.isnull().sum()


text     0
class    0
dtype: int64

In [16]:
data = data.sample(frac = 1)

In [17]:
data.head()

,text,class
9066,Immediately after the police identified the su...,0
5259,WASHINGTON (Reuters) - The White House on Sund...,1
13841,Obama does it again he put his potical agenda ...,0
18114,TRIPOLI (Reuters) - The bodies of 21 Egyptian ...,1
5771,This is disgraceful and it is an insult to eve...,0


In [18]:
data.reset_index(inplace =True)
data.drop(['index'],axis = 1, inplace =True)

In [19]:
data.columns

Index(['text', 'class'], dtype='object')

In [20]:
data.head()

,text,class
0,Immediately after the police identified the su...,0
1,WASHINGTON (Reuters) - The White House on Sund...,1
2,Obama does it again he put his potical agenda ...,0
3,TRIPOLI (Reuters) - The bodies of 21 Egyptian ...,1
4,This is disgraceful and it is an insult to eve...,0


In [21]:
import string
def wordopt(text):
    text = text.lower()
    text = re.sub('\[.*?\]', '', text)
    text = re.sub("\\W", " ", text)
    text = re.sub('https?://\S+|www\.\S+', '', text)
    text = re.sub('<.*?>+', '', text)
    text = re.sub('[%s]' % re.escape(string.punctuation), '', text)
    text = re.sub('\n', '', text)
    text = re.sub('\w*\d\w*', '', text)
    return text









In [22]:
data['text'] = data['text'].apply(wordopt)

In [23]:
x= data['text']
y = data['class']

In [24]:
from sklearn.model_selection import train_test_split

# Split the data into training and testing sets
x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=0.25, random_state=42)


In [25]:
from sklearn.feature_extraction.text import TfidfVectorizer
vectorization = TfidfVectorizer()
xv_train = vectorization.fit_transform(x_train)
xv_test = vectorization.transform(x_test)


In [47]:
xv_train 

<33658x94975 sparse matrix of type '<class 'numpy.float64'>'
	with 6886938 stored elements in Compressed Sparse Row format>

In [48]:
from sklearn.linear_model import LogisticRegression

LR = LogisticRegression()

LR.fit(xv_train,y_train)

LogisticRegression()

In [49]:
pred_lr = LR.predict(xv_test)

In [50]:
LR.score(xv_test, y_test)

0.9876114081996435

In [51]:
from sklearn.metrics import classification_report

print(classification_report(y_test, pred_lr))

              precision    recall  f1-score   support

           0       0.99      0.99      0.99      5876
           1       0.98      0.99      0.99      5344

    accuracy                           0.99     11220
   macro avg       0.99      0.99      0.99     11220
weighted avg       0.99      0.99      0.99     11220



In [30]:
from sklearn.tree import DecisionTreeClassifier
DT = DecisionTreeClassifier()
DT.fit(xv_train, y_train)

DecisionTreeClassifier()

In [31]:
pred_dt =DT.predict(xv_test)

In [32]:
DT.score(xv_test, y_test)

0.9941176470588236

In [33]:
print(classification_report(y_test, pred_dt))

              precision    recall  f1-score   support

           0       0.99      1.00      0.99      5876
           1       1.00      0.99      0.99      5344

    accuracy                           0.99     11220
   macro avg       0.99      0.99      0.99     11220
weighted avg       0.99      0.99      0.99     11220



In [34]:
from sklearn.ensemble import GradientBoostingClassifier

GB = GradientBoostingClassifier(random_state = 0)
GB.fit(xv_train, y_train)

GradientBoostingClassifier(random_state=0)

In [35]:
pred_gb = GB.predict(xv_test)

In [36]:
GB.score(xv_test, y_test)

0.99349376114082

In [37]:
print(classification_report(y_test, pred_gb))

              precision    recall  f1-score   support

           0       1.00      0.99      0.99      5876
           1       0.99      1.00      0.99      5344

    accuracy                           0.99     11220
   macro avg       0.99      0.99      0.99     11220
weighted avg       0.99      0.99      0.99     11220



In [38]:
from sklearn.ensemble import RandomForestClassifier

RF = RandomForestClassifier(random_state = 0)
RF.fit(xv_train, y_train)

RandomForestClassifier(random_state=0)

In [39]:
pred_rf = RF.predict(xv_test)

In [40]:
RF.score(xv_test, y_test)

0.9899286987522281

In [41]:
print(classification_report(y_test, pred_rf))

              precision    recall  f1-score   support

           0       0.99      0.99      0.99      5876
           1       0.99      0.99      0.99      5344

    accuracy                           0.99     11220
   macro avg       0.99      0.99      0.99     11220
weighted avg       0.99      0.99      0.99     11220



In [42]:
import pandas as pd
  # You should import the vectorization module


def output_label(n):
    if n == 0:
        return "Fake News"
    elif n == 1:
        return "Real News"

# Define the manual_testing function
def manual_testing(news):
    testing_news = {"text": [news]}
    new_def_test = pd.DataFrame(testing_news)
    new_def_test["text"] = new_def_test["text"].apply(wordopt)  # Assuming wordopt is defined elsewhere
    new_x_test = new_def_test["text"]
    new_xv_test = vectorization.transform(new_x_test)  # Assuming vectorization is defined and working correctly
    pred_LR = LR.predict(new_xv_test)
    pred_DT = DT.predict(new_xv_test)
    pred_GB = GB.predict(new_xv_test)
    pred_RF = RF.predict(new_xv_test)

    return print("\n\nLR Prediction: {} \nDT Prediction: {} \nGB Prediction: {} \nRF Prediction: {}".format(
        output_label(pred_LR[0]),
        output_label(pred_DT[0]),
        output_label(pred_GB[0]),
        output_label(pred_RF[0])
    ))



In [43]:
from flask import Flask

@app.route('/predict', methods=['POST'])
def predict():
    if request.method == 'POST':
        news_text = request.form['input_text']
        if not news_text:
            return render_template('model_page.html', prediction="Please enter news text.")
        else:
           
             manual_testing(news_text)
           
        return render_template('model_page.html', manual_testing=f"The text is classified as:")
        
if __name__ == '__main__':
     app.run( debug=True)

 * Serving Flask app '__main__'
 * Debug mode: on


 * Running on http://127.0.0.1:5000
Press CTRL+C to quit
 * Restarting with watchdog (windowsapi)


SystemExit: 1

C:\Users\Asus\AppData\Local\Programs\Python\Python311\Lib\site-packages\IPython\core\interactiveshell.py:3516: UserWarning: To exit: use 'exit', 'quit', or Ctrl-D.
  warn("To exit: use 'exit', 'quit', or Ctrl-D.", stacklevel=1)


# New Section